In [1]:
!pip install transformers datasets scikit-learn --quiet

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.optim import AdamW
import torch.nn.functional as F
from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

Load and Prepare IMDb Dataset

In [2]:

dataset = load_dataset("imdb")


train_data = dataset["train"].shuffle(seed=42)
test_data = dataset["test"]


train_df = pd.DataFrame({
    'review': train_data['text'],
    'label': train_data['label']
})
test_df = pd.DataFrame({
    'review': test_data['text'],
    'label': test_data['label']
})


train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['label'], random_state=42)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Tokenization

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


class IMDbDataset(Dataset):
    def __init__(self, df):
        self.encodings = tokenizer(list(df['review']), truncation=True, padding=True, max_length=256)
        self.labels = list(df['label'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_df)
val_dataset = IMDbDataset(val_df)
test_dataset = IMDbDataset(test_df)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Fine-tune BERT Teacher Model

In [4]:

teacher_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    do_train=True,
    do_eval=True,
    logging_dir='./logs',
    logging_steps=500,
    save_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    disable_tqdm=False
)

trainer = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-bb35c4ac7e53>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ih2363 (ih2363-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.334100
1000,0.248800


TrainOutput(global_step=1407, training_loss=0.2731656452062372, metrics={'train_runtime': 1068.4526, 'train_samples_per_second': 21.058, 'train_steps_per_second': 1.317, 'total_flos': 2959999372800000.0, 'train_loss': 0.2731656452062372, 'epoch': 1.0})

Generate Soft Labels from Teacher for Student

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)

def get_logits(model, dataset, batch_size=32):
    model.eval()
    device = next(model.parameters()).device
    loader = DataLoader(dataset, batch_size=batch_size)
    all_logits = []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            all_logits.append(outputs.logits.cpu())
    return torch.cat(all_logits)


train_teacher_logits = get_logits(teacher_model, train_dataset)

Train Student Model with Distillation

In [6]:
class IMDbDistillDataset(Dataset):
    def __init__(self, df, teacher_logits):
        self.encodings = tokenizer(list(df['review']), truncation=True, padding=True, max_length=256)
        self.labels = list(df['label'])
        self.teacher_logits = teacher_logits  # Tensor shape: (num_samples, num_classes)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['teacher_logits'] = self.teacher_logits[idx]
        return item

    def __len__(self):
        return len(self.labels)


distill_dataset = IMDbDistillDataset(train_df, train_teacher_logits)


student_model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
).to(device)

optimizer = AdamW(student_model.parameters(), lr=1e-4)

temperature = 4.0
alpha = 0.9

def distillation_loss(student_logits, teacher_logits, true_labels):
    s = F.log_softmax(student_logits / temperature, dim=1)
    t = F.softmax(teacher_logits / temperature, dim=1)
    kd = F.kl_div(s, t, reduction="batchmean") * (temperature ** 2)
    ce = F.cross_entropy(student_logits, true_labels)
    return alpha * kd + (1 - alpha) * ce

train_loader = DataLoader(distill_dataset, batch_size=32, shuffle=True)

student_model.train()
for epoch in range(3):
    total_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        t_logits = batch["teacher_logits"].to(device)

        outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
        s_logits = outputs.logits

        loss = distillation_loss(s_logits, t_logits, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} average loss: {avg:.4f}")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 average loss: 0.2944
Epoch 2 average loss: 0.1107
Epoch 3 average loss: 0.0723


Evaluation

In [7]:
def evaluate_all(model, dataset):
    model.eval()
    loader = DataLoader(dataset, batch_size=32)
    all_logits, all_labels = [], []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())

    logits = torch.cat(all_logits)
    labels = torch.cat(all_labels).numpy()
    preds = logits.argmax(dim=1).numpy()
    probs = torch.softmax(logits, dim=1)[:, 1].numpy()

    return (
        accuracy_score(labels, preds),
        f1_score(labels, preds),
        roc_auc_score(labels, probs)
    )


In [8]:
t_acc, t_f1, t_auc = evaluate_all(teacher_model, test_dataset)
s_acc, s_f1, s_auc = evaluate_all(student_model, test_dataset)

print(f"Teacher → Acc: {t_acc:.4f},  F1: {t_f1:.4f},  AUC: {t_auc:.4f}")
print(f"Student → Acc: {s_acc:.4f},  F1: {s_f1:.4f},  AUC: {s_auc:.4f}")

Teacher → Acc: 0.9162,  F1: 0.9180,  AUC: 0.9745
Student → Acc: 0.9102,  F1: 0.9078,  AUC: 0.9721


Student vs. Teacher Performance Discussion

In this implementation, the teacher model (BERT) achieved:

**Accuracy:** ~91.6%

**F1 Score:** ~91.8%

**AUC:** ~97.4%

The student model (DistilBERT), after knowledge distillation, achieved:

**Accuracy:** ~91.0%

**F1 Score:** ~90.8%

**AUC:** ~97.2%



The student model retains around ~99% of the teacher’s accuracy while being lighter and faster to run.

The small drop (~0.6% in accuracy and ~0.6% in F1) is expected because DistilBERT has fewer parameters and capacity.

AUC remains very close, suggesting that the student is still highly effective at ranking positive/negative reviews.

**Conclusion:** The distillation worked well—the student maintains excellent performance, making it a practical option when resources are limited.